# Data Processing for DV2 Datasets

### Create CSV for Choropleth (2)

Process Tuberculosis Data

In [84]:
import pandas as pd

In [101]:
# Read the CSV file
df = pd.read_csv("Disease/Tuberculosis/TB_burden_countries_2023-09-21.csv")

# Select the desired columns
selected_columns = ["country", "iso3", "year", "e_pop_num", "e_inc_num"]
df = df[selected_columns]

# Rename columns
df.rename(columns={"e_pop_num": "population", "e_inc_num": "cases"}, inplace=True)

# Add a "disease" column with the value "tuberculosis"
df["disease"] = "Tuberculosis"

# Filter rows with "year" equal to 2021
df = df[df["year"] == 2021]

# Save the filtered DataFrame as a new CSV
df.to_csv("Disease/Tuberculosis/TB_cases_2021_processed.csv", index=False)

Process Malaria Data

In [103]:
# Read the CSV file
df = pd.read_csv("Disease/Malaria/malaria_estimated_cases.csv")

# Select the columns
selected_columns = ["Location", "SpatialDimValueCode", "Period", "FactValueNumeric"]

# Rename columns
df = df[selected_columns]
df = df.rename(columns={"Location": "country", "SpatialDimValueCode": "iso3", "Period": "year", "FactValueNumeric": "cases"})

# Add a "disease" column
df["disease"] = "Malaria"

# Filter rows for the year 2021
df = df[df["year"] == 2021]

# Save the filtered DataFrame to a CSV file
df.to_csv("Disease/Malaria/malaria_cases_2021_processed.csv", index=False)

Process Measles Data

In [104]:
# Read the CSV file
df = pd.read_csv("Disease/Measles/measles_reported_cases.csv")

# Select the columns
selected_columns = ["Location", "SpatialDimValueCode", "Period", "FactValueNumeric"]

# Rename columns
df = df[selected_columns]
df = df.rename(columns={"Location": "country", "SpatialDimValueCode": "iso3", "Period": "year", "FactValueNumeric": "cases"})

# Add a "disease" column
df["disease"] = "Measles"

# Filter rows for the year 2021
df = df[df["year"] == 2021]

# Save the filtered DataFrame to a CSV file
df.to_csv("Disease/Measles/measles_cases_2021_processed.csv", index=False)

Combine Disease Datasets

In [105]:
# Read each CSV file into a separate DataFrame
df_tb = pd.read_csv("Disease/Tuberculosis/TB_cases_2021_processed.csv")
df_malaria = pd.read_csv("Disease/Malaria/malaria_cases_2021_processed.csv")
df_measles = pd.read_csv("Disease/Measles/measles_cases_2021_processed.csv")

# Merge the dataframes using an outer join to retain all rows
merged_df = pd.concat([df_tb, df_malaria, df_measles], ignore_index=True)

# Sort the merged dataframe by country, year, and disease
merged_df.sort_values(by=["country", "year", "disease"], inplace=True)

# Create a dictionary to store population values for each country and year
population_dict = {}

# Fill the population dictionary with available values
for index, row in merged_df.iterrows():
    iso3 = row["iso3"]
    year = row["year"]
    population = row["population"]
    
    # Update the population dictionary only if the value is not missing
    if not pd.isna(population):
        population_dict[(iso3, year)] = population

# Update missing population values in the dataframe
for index, row in merged_df.iterrows():
    iso3 = row["iso3"]
    year = row["year"]
    
    # Check if there is a missing population value for the current country and year
    if pd.isna(row["population"]) and (iso3, year) in population_dict:
        merged_df.at[index, "population"] = population_dict[(iso3, year)]

# Reset the index
merged_df.reset_index(drop=True, inplace=True)

# Drop rows with missing values
merged_df.dropna(inplace=True)

# Calculate cases per 100,000 population and add it as a new column
merged_df['cases_per_100k'] = (merged_df['cases'] / merged_df['population']) * 100000

# Save the merged DataFrame to a CSV file
merged_df.to_csv("Disease/disease_incidence_processed.csv", index=False)

### Make CSV for Bubble Chart

Process Tuberculosis Data (all years)

In [65]:
# Read the CSV file
tb = pd.read_csv("Disease/Tuberculosis/TB_burden_countries_2023-09-21.csv")

# Select the desired columns
selected_columns = ["country", "iso3", "year", "e_pop_num", "e_inc_100k"]
tb = tb[selected_columns]

# Rename columns
tb.rename(columns={"e_pop_num": "population", "e_inc_100k": "cases_per_100k"}, inplace=True)

# Filter rows for the year 2010 onwards
tb = tb[tb["year"] >= 2010]

# Reset the index
tb.reset_index(drop=True, inplace=True)

tb.head()

,country,iso3,year,population,cases_per_100k
0,Afghanistan,AFG,2010,28189672,189.0
1,Afghanistan,AFG,2011,29249157,189.0
2,Afghanistan,AFG,2012,30466479,189.0
3,Afghanistan,AFG,2013,31541209,189.0
4,Afghanistan,AFG,2014,32716210,189.0


Process GDP Data

In [63]:
# Read the CSV file
gdp = pd.read_csv("Economy/gdp-per-capita-worldbank.csv")

# Rename columns
gdp = gdp.rename(columns={"Entity": "country", "Code": "iso3", "Year": "year", "GDP per capita, PPP (constant 2017 international $)": "gdp_per_capita"})

# Filter rows for the year 2010 onwards
gdp = gdp[gdp["year"] >= 2010]

# Reset the index
gdp.reset_index(drop=True, inplace=True)

gdp.head()

,country,iso3,year,gdp_per_capita
0,Afghanistan,AFG,2010,2026.1638
1,Afghanistan,AFG,2011,1961.0963
2,Afghanistan,AFG,2012,2122.8308
3,Afghanistan,AFG,2013,2165.3408
4,Afghanistan,AFG,2014,2144.4497


Process Continent Mapping

In [72]:
# Read the CSV file
continents = pd.read_csv("Economy/continents-according-to-our-world-in-data.csv")

# Rename columns
continents = continents.rename(columns={"Code": "iso3", "Year": "year", "Continent": "continent"})

,Entity,iso3,year,continent
0,Abkhazia,OWID_ABK,2015,Asia
1,Afghanistan,AFG,2015,Asia
2,Akrotiri and Dhekelia,OWID_AKD,2015,Asia
3,Aland Islands,ALA,2015,Europe
4,Albania,ALB,2015,Europe


Merge Data Frames

In [81]:
# Merge the 'gdp' DataFrame with the 'tb' DataFrame based on 'iso3' and 'year'
merged_df = tb.merge(gdp[['iso3', 'year', 'gdp_per_capita']], on=['iso3', 'year'], how='inner')

# Merge the Dataframe with the 'continents' DataFrame based on 
merged_df = merged_df.merge(continents[['iso3', 'continent']], on=['iso3'], how='inner')
merged_df.head()

# Save the merged DataFrame to a CSV file
merged_df.to_csv("Economy/tuberculosis_gdp_processed.csv", index=False)

In [83]:
merged_df['population'].describe()

count    2.290000e+03
mean     3.822475e+07
std      1.437455e+08
min      1.024100e+04
25%      1.432965e+06
50%      7.365073e+06
75%      2.641980e+07
max      1.425893e+09
Name: population, dtype: float64